In [ ]:
import numpy as np

<h1> Pseudo-Random Number Generator Class'ı basit bir python generator class'ı daha sonra bu sınıfı extend alan LinearCongruentialGenerator sınıfı Mersenne-Twister algoritmasının implementasyonu için kullanılacak

In [ ]:

class PRNG(object):
    def __init__(self, X0=0):
        """X0 seed'i 0 olarak başlatıyoruz"""
        self.X0 = X0
        self.X = X0
        self.t = 0
        self.max = 0
    
    def __iter__(self):
        return self
    
    def seed(self, X0=None):
          """Seed fonksiyonu her çağırıldığında t ve X0 değeri yeniden initialize ediliyor."""
          self.t = 0
          self.X = self.X0 if X0 is None else X0
    
    def __next__(self):
        """Bir iteratör yapısıyla çalıştığımız için next fonksiyonu ile yeni bir değer üreteceğiz ilerleyen hücrelerde implementasyonu yapıldı."""
        # This default PRNG does not produce random numbers!
        self.t += 1
        return self.X
    
    def randint(self, *args, **kwargs):
        return self.__next__()

    def int_samples(self, shape=(1,)):
        """istenilen boyutta random integer sample almak için yazılan fonksiyon"""
        # return [ self.randint() for _ in range(size) ]
        return np.fromfunction(np.vectorize(self.randint), shape=shape, dtype=int)

    def rand(self, *args, **kwargs):
        """[0, 1) aralığında float bir sayı döndürür."""
        return self.randint() / float(1 + self.max)

    def float_samples(self, shape=(1,)):
        """istenilen boyutta random float sample almak için yazılan fonksiyon"""
        return np.fromfunction(np.vectorize(self.rand), shape=shape, dtype=int)
    def text_samples(self, n=1):
        """istenilen boyutta random float sample almak için yazılan fonksiyon"""
        for i in range(n):
          yield self.__next__()

<h1> linear congruential Algoritması yazıyoruz hocam. Bir önceki sınıfımızı extend alıyor. İlerleyen kod bloklarında kullanacağız.

<h2>Burada ayrıca python <b>hashlib</b> kütüphanesini import ediyoruz bu sayede sha256 algoritmasını kullanarak her bir iterasyon için <b>tohum değerini hash fonksiyonundan geçirip</b> o şekilde kullanacağız

In [ ]:
from hashlib import sha256

In [ ]:
class LinearCongruentialGenerator(PRNG):
    def __init__(self, m, a, c, X0=0):
        """init fonksiyonundan super'i çağırıp extend ettiğimiz parent generator class'ı başlatıyoruz."""
        super(LinearCongruentialGenerator, self).__init__(X0=X0)
        self.m = self.max = m
        self.a = a
        self.c = c
    
    def __next__(self):
        """Şu denklem ile çalışan next fonksiyonunu implemente ediyoruz: X_{t+1} = (a X_t + c) mod m."""
        self.t += 1
        x = self.X
        self.X = (self.a * self.X + self.c) % self.m
        return x

    


In [ ]:
""" başlangıç değerlerimiz """

m = 1 << 31 - 1 
a = 7 ** 4
c = 1

""" Bu fonksiyon ile random sayı üretecek generator fonksiyonumuzu sürekli itere ederek yeni bir sayı üretmesini sağlıyoruz """
def execute(example, nb=4):
    for t, x in enumerate(example):
        print("{:>3}th value for {.__class__.__name__} is X_t = {:>10}".format(t+1, example, x))
        if t >= nb - 1:
            break

<h1>Burada bir tohum değeri ile clasımızı new'liyoruz

In [ ]:

numner_generator = LinearCongruentialGenerator(m=m, a=a, c=c, X0=12011993)
execute(numner_generator)

<h1>sınıfımızı farklı bir tohum değeri ile seedlediğimiz zaman göründüğü gibi farklı sayılar elde ediyoruz

In [ ]:

numner_generator.seed(1201195)
execute(numner_generator)

<h1>Şimdi kurduğumuz algoritma ile elde edeceğimiz rastgele sayılardan bir matris üretip bunu pikseller haline getireceğiz

In [ ]:
""" boyut belirliyoruz ve belirlenen boyutta martis elde etmek için tanımladığımız sınıfın bir fonksiyonunu çağırıyoruz """

shape = (400, 400)
image = numner_generator.float_samples(shape)

<h1>matplotlib kütüphanesi yardımıyla elde edilen resmi çiziyoruz

In [ ]:

%matplotlib inline
import matplotlib.pyplot as plt

def showimage(image):
    plt.figure(figsize=(8, 8))
    plt.imshow(image, cmap='gray', interpolation='none')
    plt.axis('off')
    plt.show()

In [ ]:
showimage(image)

<h1>Şimdi random sayı üretecimizin ürettiği resmin Histogramını çizeceğiz bu sayede üretilen sayıların dağılımı hakkında daha iyi fikir sahibi olabiliriz

In [ ]:
def plotHistogram(example, nb=100000, bins=200):
    numbers = example.float_samples((nb,))
    plt.figure(figsize=(16, 5))
    plt.hist(numbers, bins=bins, density = True, alpha=0.8)
    plt.xlabel(" Üretilen değerler")
    plt.ylabel("Üretilen değerlerin tekrarı(ağırlığı)")
    plt.title("Üretilen Random piksellerin Histogram Grafiği".format(nb))
    plt.show()

In [ ]:
plotHistogram(numner_generator, 1000000, 200)

# Şimdi Mersenne-Twister Algoritmasını Kendi Yazdığımız Random Sayı Üreteci İçin Implemente ediyoruz

<h1>*Really random * denecek bir tohum değeri elde etmek için datetime modülünden yardım alacağız</h1>

In [ ]:
from datetime import datetime

def get_seconds():
    d = datetime.today().timestamp()
    s = 1e6 * (d - int(d))
    return int(s)
get_seconds()

806829

<h1> Şimdi linear random sayı üretecimiz ile elde ettiğimiz rastgele sayılarla Mersenne-Twister algoritmasını feed'leyeceğiz <h1>

In [ ]:
def seed_rows(example, n, w):
    return example.int_samples((n,))

def random_Mersenne_seed(n, w):
    linear = LinearCongruentialGenerator(m=(1 << 31) - 1, a=7 ** 4, c=0, X0=get_seconds())
    assert w == 32, "Error: only w = 32 was implemented"
    seed = seed_rows(linear, n, w)
    assert np.shape(seed) == (n,)
    return seed
w = 32
n = 624
example_seed = random_Mersenne_seed(n, w)

<h1> Bu adımda Mersenne-Twister algoritmasınının classını yazıyoruz burada <b>kaydırma ve xor'lama</b> işlemleri implemente edilecek </h1>

<h2>Ayrıca her __next__ iterasyonunda itere edilecek tohum değeri için <b>sha256</b> hashing algoritması kullanılacak</h2>

In [ ]:
class MersenneTwister(PRNG):
    def __init__(self, seed=None,
                 w=32, n=624, m=397, r=31,
                 a=0x9908B0DF, b=0x9D2C5680, c=0xEFC60000,
                 u=11, s=7, v=15, l=18):
        self.t = 0
        # Parametereler
        self.w = w
        self.n = n
        self.m = m
        self.r = r
        self.a = a
        self.b = b
        self.c = c
        self.u = u
        self.s = s
        self.v = v
        self.l = l
        if seed is None:
            seed = random_Mersenne_seed(n, w)
        self.X0 = seed
        self.X = np.copy(seed)
        self.max = (1 << w) - 1
        
    def __next__(self):
        self.t += 1
        #  x_t'nin ilk r kadar biti sola (x_t >> (w - r)) << (w - r)
        # {t+1}'in son w - r bit'i sağa = x & ((1 << (w - r)) - 1)
        # sonra bu sol ve sağ değerlerini bir binary vektörde birleştiriyoruz (left + right)
        left = self.X[0] >> (self.w - self.r)
        right = (self.X[1] & ((1 << (self.w - self.r)) - 1))
        x = (left << (self.w - self.r)) + right
        xw = x % 2            
        if xw == 0:
            xtilde = (x >> 1)          
        else:
            xtilde = (x >> 1) ^ self.a   
        nextx = self.X[self.m] ^ xtilde  

        oldx0 = self.X[0]         
        self.X[:-1] = self.X[1:]   # X'i bir index sola kaydırma
        # print('next : ', nextx, 'hashed : ', int(sha256(str(nextx).encode('ASCII')).hexdigest(), 16) % (10 ** 10))
        # Burada algoritmamızın tohum değerini bir sonraki iterasyona geçirmeden önce sha256 ile hashliyoruz
        nextx = int(sha256(str(nextx).encode('ASCII')).hexdigest(), 16) % (10 ** 16)
        self.X[-1]  = nextx      

        # xor'lama işlemleri
        y = nextx                      #  y = x_{t+n}
        y ^= (y >> self.u)             #  y = y ⊕ (y >> u)
        y ^= ((y << self.s) & self.b)  #  y = y ⊕ ((y << s) & b)
        y ^= ((y << self.v) & self.c)  #  y = y ⊕ ((y << v) & c)
        y ^= (y >> self.l)             #  y = y ⊕ (y >> l)
        return y

<h1> 2 hücre önce elde ettiğimiz tohum değeri ile MersenneTwister sınıfımızı örnekliyoruz

In [ ]:
mt_generator = MersenneTwister(seed=example_seed)

<h1> Şimdi yazdığımız Mersenne-Twister algoritmasını kullanarak random piksellerden oluşan bir görüntü elde edip daha sonra histogramını çizaceğiz

In [ ]:
shape = (1000, 1000)
random_numbers = mt_generator.float_samples(shape)
showimage(random_numbers)

<h1>Görüldüğü üzere gayet iyi dağılıma sahip bir random sayı üreteci implemente ettik.

In [ ]:
plotHistogram(mt_generator, 100000, 200)

<h1> Şimdi elde ettiğimiz rondom sayıları istatistiksel test için bitler haline getiriyoruz </h1>

In [ ]:
bit_array = random_numbers.reshape(-1, 1)
avg = np.average(bit_array)
bit_array = [1 if i<avg else 0 for i in bit_array]

<h1> Görüldüğü üzere üretilen 1'lerin ve 0'ların sayısı birbirine epey yakın

In [ ]:
unique, counts = np.unique(bit_array, return_counts=True)
sozluk = dict(zip(unique, counts))
print('Üretilen sıfırların sayısı: ', sozluk[0])
print('Üretilen Birlerin sayısı: ', sozluk[1])

<h1> Son olarak mersenne twister algoritmasını çıktısı <b>DES </b>Algoritmasından geçirilecek şekilde yeniden implemente ediyoruz</h1>

In [ ]:
!pip install Crypto
!pip install pycrypto

In [ ]:
# 1DES Algoritmasını kullanıyoruz
from Crypto.Cipher import DES

key = b'des12345'

des = DES.new(key, DES.MODE_ECB)

def pad(text):
        while len(text) % 8 != 0:
            text += ' '
        return text
text1 = '123456789'

padded_text = pad(text1)

encrypted_text = des.encrypt(padded_text)

print(encrypted_text)
print(des.decrypt(encrypted_text))

In [ ]:
class MersenneTwisterWithDes(PRNG):
    def __init__(self, seed=None,
                 w=32, n=624, m=397, r=31,
                 a=0x9908B0DF, b=0x9D2C5680, c=0xEFC60000,
                 u=11, s=7, v=15, l=18):
        self.t = 0
        # Parametereler
        self.w = w
        self.n = n
        self.m = m
        self.r = r
        self.a = a
        self.b = b
        self.c = c
        self.u = u
        self.s = s
        self.v = v
        self.l = l
        if seed is None:
            seed = random_Mersenne_seed(n, w)
        self.X0 = seed
        self.X = np.copy(seed)
        self.max = (1 << w) - 1
        
    def __next__(self):
        self.t += 1
        #  x_t'nin ilk r kadar biti sola (x_t >> (w - r)) << (w - r)
        # {t+1}'in son w - r bit'i sağa = x & ((1 << (w - r)) - 1)
        # sonra bu sol ve sağ değerlerini bir binary vektörde birleştiriyoruz (left + right)
        left = self.X[0] >> (self.w - self.r)
        right = (self.X[1] & ((1 << (self.w - self.r)) - 1))
        x = (left << (self.w - self.r)) + right
        xw = x % 2            
        if xw == 0:
            xtilde = (x >> 1)          
        else:
            xtilde = (x >> 1) ^ self.a   
        nextx = self.X[self.m] ^ xtilde  

        oldx0 = self.X[0]         
        self.X[:-1] = self.X[1:]   # X'i bir index sola kaydırma
        # print('next : ', nextx, 'hashed : ', int(sha256(str(nextx).encode('ASCII')).hexdigest(), 16) % (10 ** 10))
        # Burada algoritmamızın tohum değerini bir sonraki iterasyona geçirmeden önce sha256 ile hashliyoruz
        nextx = int(sha256(str(nextx).encode('ASCII')).hexdigest(), 16) % (10 ** 16)
        self.X[-1]  = nextx      

        # xor'lama işlemleri
        y = nextx                      #  y = x_{t+n}
        y ^= (y >> self.u)             #  y = y ⊕ (y >> u)
        y ^= ((y << self.s) & self.b)  #  y = y ⊕ ((y << s) & b)
        y ^= ((y << self.v) & self.c)  #  y = y ⊕ ((y << v) & c)
        y ^= (y >> self.l)             #  y = y ⊕ (y >> l)
        padded_text = pad(str(bytes(str(y), encoding='utf-8')))
        y = des.encrypt(padded_text)
        return y

In [ ]:
mtwd_generator = MersenneTwisterWithDes(seed=example_seed)

In [ ]:
n = 100
random_bytes = list(mtwd_generator.text_samples(n))
random_encrypted_texts = [i.hex() for i in random_bytes]

In [ ]:
print(random_encrypted_texts)

<h1> Örnek olması için random dizi içerisinden bir text'i encrypt ederek Mersenne Twister algoritmasının gerçekte hangi sayıyı ürettiğine bakıyoruz </h1>

In [ ]:
byte_arr = bytearray.fromhex(random_encrypted_texts[3])
string = str(des.decrypt(bytes(byte_arr))).strip().replace(" ", "")
print(string[4:-2])